**RQ 1**
---

**How effective are awe-inspiring scenes in eliciting the experience of awe compared to neutral scenes?**

- H1.1: Awe-inspiring scenes will elicit significantly higher levels of awe compared to neutral scenes. 
    - **ToDo:** Analyse AWE-S scores, especially comparing neutral vs. awe scenes (paired t-test) and abstract/natural/human-made (rm ANOVA)
- H1.2: Participants will report a greater sense of presence in awe-inspiring scenes compared to neutral scenes, and this increased presence will be positively correlated with the level of awe experienced.
    - **ToDo:** Analyse IPQ scores, look at correlation between IPQ and Awe-S results

Get from data: 
- AWE-S scores
- IPQ scores

**Boferroni correction for multiple comparisons** \
For Hypothesis 1 we have \
H1.1: 45 + 1 + 3 \
H1.2: 45 + 1 + 3 + 1 \
= 99

In [3]:
bonferroniCorrection = 99

**Import Statements**

In [4]:
import pingouin as pg
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.stats import shapiro

**Load the Data**

In [8]:
questionnaireData = pd.read_csv('processed data/combined_questionnaire_dataLikert.csv')
conditions = ['AbstractAwe', 'Borealis', 'Church', 'Forest', 'Mountain', 'Neutral1', 'Neutral3', 'Space', 'Underwater', 'Waterfall']

awe_s_columns = [col for col in questionnaireData.columns if col.startswith("AWE-S_Total")]
awe_s_data = questionnaireData.melt(id_vars=['participantID'], value_vars=awe_s_columns, var_name='Condition', value_name='Score')
awe_s_total = questionnaireData[awe_s_columns]

ipq_columns = [col for col in questionnaireData.columns if col.startswith("IPQ_Total")]
ipq_data = questionnaireData.melt(id_vars=['participantID'], value_vars=ipq_columns, var_name='Condition', value_name='Score')
ipq_total = questionnaireData[ipq_columns]

**Show Descriptive statistics**

In [9]:
descriptives_awes = awe_s_total.describe().T
descriptives_awes['N'] = awe_s_total.count()
descriptives_awes['Median'] = awe_s_total.median()
descriptives_awes['Shapiro-p'] = awe_s_total.apply(lambda x: round(shapiro(x.dropna())[1], 3))
descriptives_awes = descriptives_awes[['N', 'mean', 'Median', 'std', 'Shapiro-p']]
all_awe_scores = awe_s_total.values.flatten()
all_awe_scores = all_awe_scores[~pd.isnull(all_awe_scores)]

descriptives_ipq = ipq_total.describe().T
descriptives_ipq['N'] = ipq_total.count()
descriptives_ipq['Median'] = ipq_total.median()
descriptives_ipq['Shapiro-p'] = ipq_total.apply(lambda x: round(shapiro(x.dropna())[1], 3))
descriptives_ipq = descriptives_ipq[['N', 'mean', 'Median', 'std', 'Shapiro-p']]
all_ipq_scores = ipq_total.values.flatten()
all_ipq_scores = all_ipq_scores[~pd.isnull(all_ipq_scores)]

descriptives_awes_sorted = descriptives_awes.sort_values(by='mean', ascending=False)
descriptives_ipq_sorted = descriptives_ipq.sort_values(by='mean', ascending=False)

print("AWE-S Descriptives:")
print(descriptives_awes_sorted)

print("\nIPQ Descriptives:")
print(descriptives_ipq_sorted)

AWE-S Descriptives:
                          N  mean  Median   std  Shapiro-p
AWE-S_Total_Space        62 4.295   4.354 1.081      0.067
AWE-S_Total_Underwater   62 3.946   4.208 1.188      0.260
AWE-S_Total_Borealis     60 3.900   4.042 1.189      0.678
AWE-S_Total_Mountain     62 3.392   3.417 1.138      0.444
AWE-S_Total_AbstractAwe  62 3.359   3.354 1.085      0.331
AWE-S_Total_Forest       62 3.345   3.417 1.072      0.217
AWE-S_Total_Waterfall    60 3.272   3.229 1.087      0.202
AWE-S_Total_Church       62 3.192   3.125 1.328      0.147
AWE-S_Total_Neutral1     62 2.212   2.021 1.038      0.000
AWE-S_Total_Neutral3     62 2.195   1.854 1.046      0.000

IPQ Descriptives:
                        N  mean  Median   std  Shapiro-p
IPQ_Total_Space        62 4.327   4.267 0.742      0.115
IPQ_Total_Borealis     60 4.264   4.267 0.739      0.011
IPQ_Total_Church       62 4.220   4.267 0.756      0.001
IPQ_Total_Underwater   62 4.195   4.233 0.697      0.132
IPQ_Total_Mountain     62 4

*mayority of data normally distributed, so parametric tests are used*

**Hypothesis 1.1**
---
1. Do repeated measures ANOVA between Awe-S scores of all scenes, look at post hoc analysis
2. Focus Neutral scenes: put Neutral scene scores in one pot, all other scenes in the other and do paired t-test
3. Groups: group by "human_made", "natural" and "abstract" (leaving out neutral scenes) and perform rm ANOVA

**1. RM ANOVA and Post Hoc between Awe-S scores of all scenes**

In [10]:
# 1 RM ANOVA

for condition in conditions:
    awe_s_data['Condition'] = awe_s_data['Condition'].str.replace(f"AWE-S_Total_{condition}", condition)

pd.options.display.float_format = '{:.3f}'.format
rmanova_awe = pg.rm_anova(dv='Score', within='Condition', subject='participantID', data=awe_s_data, detailed=True)

rmanova_awe['p-unc'] = rmanova_awe['p-unc'] * bonferroniCorrection 
rmanova_awe['p-unc'] = rmanova_awe['p-unc'].clip(upper=1)

print("Repeated Measures ANOVA Result:")
print(rmanova_awe)

#sphericity violated, so we do gg correction
gg_corrected_df1 = rmanova_awe['eps'][0] * rmanova_awe['DF'][0]
gg_corrected_df2 = rmanova_awe['eps'][0] * rmanova_awe['DF'][1]
f_stat = rmanova_awe['F'][0]

print("\n Assumption of sphericity violated, Greenhouse-Geisser Correction used.")
print(f"\033[1m F({gg_corrected_df1:.3f}, {gg_corrected_df2:.3f}) = {f_stat:.3f}, p = {rmanova_awe['p-GG-corr'][0] * 99:.3f}, η2 = {rmanova_awe['ng2'][0]:.3f}")

# Posthoc analysis
posthoc_awe = pg.pairwise_tests(dv='Score', within='Condition', subject='participantID', data=awe_s_data, padjust='bonf')
posthoc_awe_filtered = posthoc_awe.drop(columns=['Contrast', 'Paired', 'Parametric', 'alternative', 'p-adjust', 'BF10'])

# visualized 
def highlight_significant_pvalue(row):
    color = 'background-color: green' if row['p-corr'] < 0.05 else ''
    return [color] * len(row)

posthoc_awe_styled = posthoc_awe_filtered.style.apply(highlight_significant_pvalue, axis=1)
posthoc_awe_styled

Repeated Measures ANOVA Result:
      Source      SS   DF     MS      F  p-unc  p-GG-corr   ng2   eps  \
0  Condition 245.166    9 27.241 58.999  0.000      0.000 0.246 0.665   
1      Error 241.014  522  0.462    NaN    NaN        NaN   NaN   NaN   

  sphericity  W-spher  p-spher  
0      False    0.148    0.000  
1        NaN      NaN      NaN  

 Assumption of sphericity violated, Greenhouse-Geisser Correction used.
 F(5.989, 347.387) = 58.999, p = 0.000, η2 = 0.246


,A,B,T,dof,p-unc,p-corr,hedges
0,AbstractAwe,Borealis,-5.296880,58.000000,0.000002,0.000085,-0.486775
1,AbstractAwe,Church,1.265912,58.000000,0.210606,1.000000,0.134664
2,AbstractAwe,Forest,0.072139,58.000000,0.942739,1.000000,0.008382
3,AbstractAwe,Mountain,-0.221980,58.000000,0.825109,1.000000,-0.023112
4,AbstractAwe,Neutral1,7.175629,58.000000,0.000000,0.000000,1.027856
5,AbstractAwe,Neutral3,7.943921,58.000000,0.000000,0.000000,1.073534
6,AbstractAwe,Space,-7.496652,58.000000,0.000000,0.000000,-0.856921
7,AbstractAwe,Underwater,-5.209211,58.000000,0.000003,0.000118,-0.503339
8,AbstractAwe,Waterfall,0.730251,58.000000,0.468176,1.000000,0.066285
9,Borealis,Church,7.156974,58.000000,0.000000,0.000000,0.569492


**2. Focus Neutral scenes: put Neutral scene scores in one pot, all other scenes in the other and do paired t-test**


In [14]:
# Combine 'Neutral1' and 'Neutral3' into one 'Neutral' condition, and the rest of the scenes into 'Awe' condition

awe_s_data['Condition_Group'] = awe_s_data['Condition'].apply(lambda x: 'Neutral' if x in ['Neutral1', 'Neutral3'] else 'Awe')
neutral_group_scores = awe_s_data[awe_s_data['Condition_Group'] == 'Neutral'].groupby('participantID')['Score'].mean()
awe_group_scores = awe_s_data[awe_s_data['Condition_Group'] == 'Awe'].groupby('participantID')['Score'].mean()

t_test = pg.ttest(neutral_group_scores, awe_group_scores, paired=True)

# Effect size: maybe use r not d?
r = (t_test['T'].iloc[0]**2 / (t_test['T'].iloc[0]**2 + t_test['dof'].iloc[0]))**0.5
print(f"\nEffect size r: {r:.3f}")

# print out nicely
print(f"\n\033[1m t({int(t_test['dof'].iloc[0])}) = {t_test['T'].iloc[0]:.3f}, p = {t_test['p-val'].iloc[0]*bonferroniCorrection:.3f}, d = {t_test['cohen-d'].iloc[0]:.2f}")


Effect size r: 0.851

 t(61) = -12.657, p = 0.000, d = 1.41


**3. Groups: group by "human_made", "natural" and "abstract" and perform rm ANOVA**

In [15]:
#RM ANOVA between natural, human_made and abstract scenes, leaving out Neutral Scenes
groups = {
    'Human_Made': ['Church'],
    'Natural': ['Underwater', 'Waterfall', 'Space', 'Mountain', 'Forest', 'Borealis'],
    'Abstract': ['AbstractAwe']
}

def assign_group(condition):
    for group, conditions in groups.items():
        if condition in conditions:
            return group
    return None

awe_s_data['Group'] = awe_s_data['Condition'].apply(assign_group)
awe_s_data_noNeutral = awe_s_data.dropna(subset=['Group'])

natural_means = (
    awe_s_data[awe_s_data['Group'] == 'Natural']
    .groupby('participantID', as_index=False)['Score']
    .mean()
)
natural_means['Group'] = 'Natural'
awe_s_data_noNeutral = awe_s_data[awe_s_data['Group'] != 'Natural']
awe_s_data_noNeutral = pd.concat([awe_s_data_noNeutral, natural_means])
awe_s_data_noNeutral = awe_s_data.dropna(subset=['Group'])

rmanova_groups = pg.rm_anova(dv='Score', within='Group', subject='participantID', data=awe_s_data_noNeutral, detailed=True)
print("Repeated Measures ANOVA Result:")
print(rmanova_groups)

#sphericity violated
gg_corrected_df1 = rmanova_groups['DF'][0] * rmanova_groups['eps'][0]
gg_corrected_df2 = rmanova_groups['DF'][1] * rmanova_groups['eps'][0]
f_stat = rmanova_groups['F'][0]

print("\n Assumption of sphericity violated, Greenhouse-Geisser Correction used.")
print(f"\033[1m F({gg_corrected_df1:.3f}, {gg_corrected_df2:.3f}) = {f_stat:.3f}, p = {rmanova_groups['p-GG-corr'][0]*99:.3f}, η2 = {rmanova_groups['ng2'][0]:.3f}\033[0m")

# Post-hoc pairwise tests
posthoc_groups = pg.pairwise_tests(dv='Score', within='Group', subject='participantID', data=awe_s_data_noNeutral, padjust='bonf')
posthoc_groups_filtered = posthoc_groups.drop(columns=['Contrast', 'Paired', 'Parametric', 'alternative', 'p-adjust', 'BF10'])
posthoc_groups_filtered['p-unc'] = posthoc_groups_filtered['p-unc'] *bonferroniCorrection
posthoc_groups_filtered['p-unc'] = posthoc_groups_filtered['p-unc'].clip(upper=1)
print("\nPost-Hoc Pairwise Tests:")
print(posthoc_groups_filtered)

Repeated Measures ANOVA Result:
  Source     SS   DF    MS      F  p-unc  p-GG-corr   ng2   eps sphericity  \
0  Group  8.383    2 4.192 12.437  0.000      0.000 0.034 0.806      False   
1  Error 41.117  122 0.337    NaN    NaN        NaN   NaN   NaN        NaN   

   W-spher  p-spher  
0    0.759    0.000  
1      NaN      NaN  

 Assumption of sphericity violated, Greenhouse-Geisser Correction used.
 F(1.612, 98.336) = 12.437, p = 0.006, η2 = 0.034

Post-Hoc Pairwise Tests:
            A           B      T    dof  p-unc  p-corr  hedges
0    Abstract  Human_Made  1.317 61.000  1.000   0.579   0.137
1    Abstract     Natural -3.935 61.000  0.021   0.001  -0.328
2  Human_Made     Natural -5.414 61.000  0.000   0.000  -0.433


**Hypothesis 1.2**
---

1. Do repeated measures ANOVA between IPQ scores of all scenes, look at post hoc analysis
2. Look at pairwise t-test between IPQ scores of neutral and awe-inspiring stimuli
3. Look at human-made, natural and abstract gruops (rm ANOVA)
4. Do correlation analysis beween Awe-S and IPQ scores

**1. RM ANOVA between IPQ scores of all scenes**

In [4]:
# 1 RM ANOVA
for condition in conditions:
    ipq_data['Condition'] = ipq_data['Condition'].str.replace(f"IPQ_Total_{condition}", condition)

pd.options.display.float_format = '{:.3f}'.format
rmanova_ipq = pg.rm_anova(dv='Score', within='Condition', subject='participantID', data=ipq_data, detailed=True)
print("Repeated Measures ANOVA Result:")
print(rmanova_ipq)

#sphericity violated, so we do gg correction
gg_corrected_df1_ipq = rmanova_ipq['eps'][0] * rmanova_ipq['DF'][0]
gg_corrected_df2_ipq = rmanova_ipq['eps'][0] * rmanova_ipq['DF'][1]
f_stat_ipq = rmanova_ipq['F'][0]

print("\n Assumption of sphericity violated, Greenhouse-Geisser Correction used.")
print(f"\033[1m F({gg_corrected_df1_ipq:.3f}, {gg_corrected_df2_ipq:.3f}) = {f_stat_ipq:.3f}, p = {rmanova_ipq['p-GG-corr'][0]*99:.3f}, η2 = {rmanova_ipq['ng2'][0]:.3f}\033[0m")

# Posthoc analysis
posthoc_ipq = pg.pairwise_tests(dv='Score', within='Condition', subject='participantID', data=ipq_data, padjust='None')
posthoc_ipq_filtered = posthoc_ipq.drop(columns=['Contrast', 'Paired', 'Parametric', 'alternative', 'BF10'])

#Bonferroni Correction
posthoc_ipq_filtered['p-corr'] = posthoc_ipq_filtered['p-unc'] * bonferroniCorrection
posthoc_ipq_filtered['p-corr'] = posthoc_ipq_filtered['p-corr'].clip(upper=1)

# visualized  posthoc
def highlight_significant_pvalue(row):
    color = 'background-color: green' if row['p-corr'] < 0.05 else ''
    return [color] * len(row)

posthoc_ipq_styled = posthoc_ipq_filtered.style.apply(highlight_significant_pvalue, axis=1)
posthoc_ipq_styled

Repeated Measures ANOVA Result:
      Source      SS   DF    MS      F  p-unc  p-GG-corr   ng2   eps  \
0  Condition  25.349    9 2.817 13.375  0.000      0.000 0.069 0.768   
1      Error 109.927  522 0.211    NaN    NaN        NaN   NaN   NaN   

  sphericity  W-spher  p-spher  
0      False    0.208    0.000  
1        NaN      NaN      NaN  

 Assumption of sphericity violated, Greenhouse-Geisser Correction used.
 F(6.913, 400.928) = 13.375, p = 0.000, η2 = 0.069


,A,B,T,dof,p-unc,hedges,p-corr
0,AbstractAwe,Borealis,-5.558016,58.000000,0.000001,-0.457705,0.000071
1,AbstractAwe,Church,-3.694697,58.000000,0.000489,-0.394872,0.048452
2,AbstractAwe,Forest,-0.573886,58.000000,0.568264,-0.067918,1.000000
3,AbstractAwe,Mountain,-1.056911,58.000000,0.294935,-0.125949,1.000000
4,AbstractAwe,Neutral1,3.221297,58.000000,0.002094,0.363734,0.207289
5,AbstractAwe,Neutral3,-0.217281,58.000000,0.828752,-0.024511,1.000000
6,AbstractAwe,Space,-4.465357,58.000000,0.000037,-0.520437,0.003711
7,AbstractAwe,Underwater,-3.381107,58.000000,0.001297,-0.399248,0.128446
8,AbstractAwe,Waterfall,-0.171819,58.000000,0.864178,-0.020004,1.000000
9,Borealis,Church,0.619565,58.000000,0.537971,0.058090,1.000000


**2. IPQ: neutral vs. awe-inspiring**

In [8]:
#combine 'Neutral1' and 'Neutral3' into one 'Neutral' condition, and the rest of the scenes into 'Awe' condition, then compare
ipq_data['Condition_Group'] = ipq_data['Condition'].apply(lambda x: 'Neutral' if x in ['Neutral1', 'Neutral3'] else 'Awe')
ipq_neutral_scores = ipq_data[ipq_data['Condition_Group'] == 'Neutral'].groupby('participantID')['Score'].mean()
ipq_other_scores = ipq_data[ipq_data['Condition_Group'] == 'Awe'].groupby('participantID')['Score'].mean()

ipq_t_test = pg.ttest(ipq_neutral_scores, ipq_other_scores, paired=True)

print("\nIPQ Paired T-Test Result:")
print(f"\n\033[1m t({int(ipq_t_test['dof'].iloc[0])}) = {ipq_t_test['T'].iloc[0]:.3f}, p = {ipq_t_test['p-val'].iloc[0]*bonferroniCorrection:.3f}, d = {ipq_t_test['cohen-d'].iloc[0]:.3f}")


IPQ Paired T-Test Result:

 t(61) = -5.830, p = 0.000, d = 0.474


**3. Natural, human-made and abstract groups**

In [26]:
#RM ANOVA between natural, human_made and abstract scenes
ipq_data['Group'] = ipq_data['Condition'].apply(assign_group)
ipq_data_noNeutral = ipq_data.dropna(subset=['Group'])

rmanova_groups_ipq = pg.rm_anova(dv='Score', within='Group', subject='participantID', data=ipq_data_noNeutral, detailed=True)
print("Repeated Measures ANOVA Result:")
print(rmanova_groups_ipq)

#sphericity violated
corr_df1 = rmanova_groups_ipq['DF'][0] * rmanova_groups_ipq['eps'][0]
corr_df2 = rmanova_groups_ipq['DF'][1] * rmanova_groups_ipq['eps'][0]
f_stat = rmanova_groups_ipq['F'][0]

print("\n Assumption of sphericity violated, Greenhouse-Geisser Correction used.")
print(f"\033[1m F({corr_df1:.3f}, {corr_df2:.3f}) = {f_stat:.3f}, p = {rmanova_groups_ipq['p-GG-corr'][0]*bonferroniCorrection:.3f}, η2 = {rmanova_groups_ipq['ng2'][0]:.3f}\033[0m ")

# Post-hoc pairwise tests
ipq_posthoc_groups = pg.pairwise_tests(dv='Score', within='Group', subject='participantID', data=ipq_data_noNeutral, padjust='bonf')
ipq_posthoc_groups_filtered = ipq_posthoc_groups.drop(columns=['Contrast', 'Paired', 'Parametric', 'alternative', 'p-adjust', 'BF10'])
print("\nPost-Hoc Pairwise Tests:")
print(ipq_posthoc_groups_filtered)

NameError: name 'assign_group' is not defined

**4. Correlations IPQ vs AWE-S**

In [27]:
# correlations
def calculate_correlations_apa(merged_data, awe_s_data, ipq_data):
    apa_results = {}
    
    # correlation for each condition
    for condition in conditions:
        condition_data = merged_data[merged_data['Condition'] == condition]
        # pearsons correlation coefficient
        corr_result = pg.corr(condition_data['Score_AWE'], condition_data['Score_IPQ'])
        
        r_value = corr_result['r'].values[0]
        p_value = corr_result['p-val'].values[0]
        df = len(condition_data) - 2
        
        apa_result = f"r({df}) = {r_value:.3f}, p = {p_value:.3f}"
        apa_results[condition] = apa_result
        print(f"Correlation between Awe-S and IPQ scores for {condition}: {apa_result}")
    
    #correlation across all conditions combined
    all_data_corr_result = pg.corr(merged_data['Score_AWE'], merged_data['Score_IPQ'])
    all_r_value = all_data_corr_result['r'].values[0]
    all_p_value = all_data_corr_result['p-val'].values[0]
    all_df = len(merged_data) - 2
    
    all_data_apa_result = f"r({all_df}) = {all_r_value:.3f}, p = {all_p_value*bonferroniCorrection:.3f}"
    apa_results['All_Conditions'] = all_data_apa_result
    print(f"\033[1m Correlation between Awe-S and IPQ scores across all conditions combined: {all_data_apa_result}")
    
    return apa_results

merged_data = pd.merge(
    awe_s_data, ipq_data,
    on=['participantID', 'Condition'],
    suffixes=('_AWE', '_IPQ')
)
    
correlations_apa = calculate_correlations_apa(merged_data, awe_s_data, ipq_data)

Correlation between Awe-S and IPQ scores for AbstractAwe: r(60) = 0.580, p = 0.000
Correlation between Awe-S and IPQ scores for Borealis: r(60) = 0.757, p = 0.000
Correlation between Awe-S and IPQ scores for Church: r(60) = 0.612, p = 0.000
Correlation between Awe-S and IPQ scores for Forest: r(60) = 0.581, p = 0.000
Correlation between Awe-S and IPQ scores for Mountain: r(60) = 0.734, p = 0.000
Correlation between Awe-S and IPQ scores for Neutral1: r(60) = 0.576, p = 0.000
Correlation between Awe-S and IPQ scores for Neutral3: r(60) = 0.539, p = 0.000
Correlation between Awe-S and IPQ scores for Space: r(60) = 0.618, p = 0.000
Correlation between Awe-S and IPQ scores for Underwater: r(60) = 0.648, p = 0.000
Correlation between Awe-S and IPQ scores for Waterfall: r(60) = 0.598, p = 0.000
 Correlation between Awe-S and IPQ scores across all conditions combined: r(618) = 0.624, p = 0.000
